#**Lab 4 - Part 1** (30 Marks)

Group 9:

Danielle do Val Goncalves Tudeia - W0823569

Fernanda Barbieri de Camargo - W0825882

Jonathan Alberto Calle Zuniga - W0825959

Jonathan Oteh - W0775057

Luis Ramirez Fernandez - W0811391

## Hyperparameter tuning by hand (Manual Method)

In this section, you will create a fully-connected neural network and then tune the hyperparameters by hand. To mix things up a little bit, we will use the **[Shoe, Sandal, and Boot](https://www.kaggle.com/datasets/hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images)** dataset from Kaggle.

### Import statements




In [2]:
# TO DO: collect all needed import statements here

# Data Overview
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Flatten



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Get the data (10 Marks)

In [3]:
# TO DO: use 'image_dataset_from_directory' (making sure to also set the 'validation_split', 'subset', and 'seed' arguments)
#        to create training and validation datasets. Set the appropriate image size by consulting the data documentation

base_dir = pathlib.Path('/content/drive/MyDrive/Lab4ML2/Shoe vs Sandal vs Boot Dataset')

image_size = (224, 224)
batch_size = 32


In [4]:
# prompt: calculate train_ds, val_ds

train_ds = image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
)


Found 15010 files belonging to 3 classes.
Using 12008 files for training.
Found 15010 files belonging to 3 classes.
Using 3002 files for validation.


In [5]:
for imgs, labs in train_ds:
  print(imgs.shape)
  print(labs.shape)
  break

(32, 224, 224, 3)
(32,)


### Set hyperparameter values to test

In [6]:
# TO DO: create lists with 3 choices for each hyperparameter

neuron_choices = [16, 32, 64]
activations = ['relu', 'sigmoid', 'tanh']
optimizers = ['adam', 'sgd', 'rmsprop']


### Create a function to build a model (5 Marks)

Since we will need to create many models with the same structure, just with different hyperparameter values, we should create a function that will do this for us.

In [7]:
# TO DO: create a function to build a model using the functional API that includes rescaling, a single hidden layer, and an appropriate output layer

def build_model(neurons, activation, optimizer):
  inputs = keras.Input(shape=(224, 224, 3))
  x = Flatten()(inputs)
  x = Rescaling(1./255)(x)
  x = Dense(neurons, activation=activation)(x)
  outputs = Dense(3, activation='softmax')(x)
  model = keras.Model(inputs, outputs)

  return model

### Test all combinations and save the best model (15 Marks)

For each set of hyperparameters we will want to train the model and then save the best version of that model after training.

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

path = str('/content/drive/MyDrive/Lab4ML2/best_model.h5')

for neurons in neuron_choices:
    for activation in activations:
        for optimizer in optimizers:
            model = build_model(neurons, activation, optimizer) # build model
            model.compile(
                optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']) # compile model

            callback = ModelCheckpoint(
                filepath=path,
                monitor="val_loss",
                save_best_only=True,
                save_weights_only=True) #create callback (ModelCheckpoint)

            model.fit(
                train_ds,
                validation_data=val_ds,
                epochs=2,
                callbacks=[callback],
                verbose=0) #train model

            # load the model that was saved as part of the callback

            test_model = build_model(neurons, activation, optimizer) # create a model to test

            test_model.load_weights(path) #load the saved weights

            test_model.compile(
                loss = 'sparse_categorical_crossentropy',
                optimizer = optimizer,
                metrics = ['accuracy']) #compile model

            # calculate the validation loss and accuracy by evaluating the best model on the validation set
            val_loss, val_accu = test_model.evaluate(val_ds, verbose = 0)

            # print out hyperparameters and validation loss and accuracy

            print(f"Neurons: {neurons} / Act Function = {activation} / Optimizer: {optimizer} / Val_loss: {val_loss:.3f} / Val_accu: {val_accu:.3f}")


Neurons: 16 / Act Function = relu / Optimizer: adam / Val_loss: 1.099 / Val_accu: 0.332
Neurons: 16 / Act Function = relu / Optimizer: sgd / Val_loss: 1.099 / Val_accu: 0.333
Neurons: 16 / Act Function = relu / Optimizer: rmsprop / Val_loss: 1.099 / Val_accu: 0.333
Neurons: 16 / Act Function = sigmoid / Optimizer: adam / Val_loss: 1.099 / Val_accu: 0.333
Neurons: 16 / Act Function = sigmoid / Optimizer: sgd / Val_loss: 0.961 / Val_accu: 0.708
Neurons: 16 / Act Function = sigmoid / Optimizer: rmsprop / Val_loss: 1.099 / Val_accu: 0.329
Neurons: 16 / Act Function = tanh / Optimizer: adam / Val_loss: 1.099 / Val_accu: 0.329
Neurons: 16 / Act Function = tanh / Optimizer: sgd / Val_loss: 1.099 / Val_accu: 0.338
Neurons: 16 / Act Function = tanh / Optimizer: rmsprop / Val_loss: 1.099 / Val_accu: 0.329
Neurons: 32 / Act Function = relu / Optimizer: adam / Val_loss: 2.181 / Val_accu: 0.839
Neurons: 32 / Act Function = relu / Optimizer: sgd / Val_loss: 1.099 / Val_accu: 0.333
Neurons: 32 / Act 